In [1]:
# Load Libraries
import warnings
from datetime import datetime

import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, LSTM, Input
from keras.layers.core import Dense, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
def lstm(max_len=73, emb_dim=32, max_vocab_len=40, W_reg=regularizers.l2(1e-4)):
    """LSTM model with the Keras Sequential model"""

    input = Input(shape=(max_len,), dtype='int32', name='lstm_input')
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(input)
    emb = Dropout(0.5)(emb)
    lstm = LSTM(128)(emb)
    lstm = Dropout(0.5)(lstm)
    output = Dense(21, activation='softmax', name='lstm_output')(lstm)
    model = Model(input=[input], output=[output])

    return model

In [3]:
# Load data using model preprocessor
x_train, x_test, y_train, y_test = Preprocessor.load_data()

In [4]:
# Define Deep Learning Model
model_name = "LSTM"
model = lstm()

''' Training phrase '''
epochs = 5
batch_size = 64
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1149965 samples, validate on 142131 samples
Epoch 1/5
1149965/1149965 [==============================] - 981s 853us/step - loss: 0.7628 - accuracy: 0.7766 - categorical_accuracy: 0.7057 - precision: 0.8709 - recall: 0.7051 - fmeasure: 0.7762 - val_loss: 0.4641 - val_accuracy: 0.8646 - val_categorical_accuracy: 0.7816 - val_precision: 0.9264 - val_recall: 0.8232 - val_fmeasure: 0.8713
Epoch 2/5
1149965/1149965 [==============================] - 990s 861us/step - loss: 0.4688 - accuracy: 0.8634 - categorical_accuracy: 0.8060 - precision: 0.9217 - recall: 0.8207 - fmeasure: 0.8678 - val_loss: 0.3346 - val_accuracy: 0.9038 - val_categorical_accuracy: 0.8249 - val_precision: 0.9449 - val_recall: 0.8701 - val_fmeasure: 0.9056
Epoch 3/5
1149965/1149965 [==============================] - 980s 852us/step - loss: 0.3569 - accuracy: 0.8981 - categorical_accuracy: 0.8386

In [5]:
model_json = model.to_json()
with open("./saved_model/lstm.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/lstm.h5")